In [1]:
from app.core.config import settings
from app.core.enums import LessonTypeEnum, UserType
from app.db.session import SessionLocal, engine, get_db, Base
from app.db.init_db import init_db, init_test_db
from app.schemas.credit import CreditCreate
from app.schemas.user import UserCreate
from sqlalchemy.ext.declarative import declarative_base
from app.crud.user import crud_user
from app.crud.credit import crud_credit
from app.models.credit import Credit
from app.models.user import User


In [2]:
db = SessionLocal()
Base.metadata.create_all(bind=engine)
init_test_db(db)


### 코드 저장

In [ ]:
# 수강 완료 시 크레딧 사용 완료
credit_id = 12
crud_credit.update_is_used_to_true(db, credit_id)

In [9]:
# 결제 시, 혹은 외상 신청시 크레딧 생성
instructor_id = 4
student_id = 5
crud_credit.create(
    db,
    obj_in=CreditCreate(
        credit_type=LessonTypeEnum.typePostPay,
        own_student_id=student_id,
        target_instructor_id=instructor_id,
    ),
)


In [6]:
# instructor 번호 왔을 때 학생 리스트 보내주기
def student_list_instructor():
    instructor_id = 4
    crud_credit.get_all_students_unused_credits_to_instructor(
        db, instructor_id
    )


# 학생별 크레딜 리스트
def credit_list_by_student():
    instructor_id = 4
    student_id = 5
    crud_credit.get_student_unused_credits_to_instructor(
        db, instructor_id, student_id
    )
